# DEEP LEARNING FOR SKIN LESIONS

This is our primary jupyter notebook for this project.

Imports:

In [12]:
import numpy as np
import pandas as pd


# import sklearn stuff
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split


# set up for plotting figures in the notebook
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

#for image processing
from glob import glob

Phase 1: Load in the data


In [ ]:
#NOTE: THIS DOWLOAD CODE IS FROM KAGGLEHUB WEBSITE

# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "HAM10000_metadata.csv"

# Load the latest version
df = kagglehub.dataset_load( #changed from load_dataaset cause apparently it's going to be deprecated lol
  KaggleDatasetAdapter.PANDAS,
  "kmader/skin-cancer-mnist-ham10000",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

# split off 30% to be validation/test sets
train_df, temp_df = train_test_split(df, test_size=0.30, stratify=df['dx'], random_state=42)

# split this set (30% of original data) into halves, so 15% val, 15% test
val_df, test_df = train_test_split(temp_df, test_size=0.50, stratify=temp_df['dx'], random_state=42)

# print(len(train_df), len(val_df), len(test_df))

First 5 records:      lesion_id      image_id   dx dx_type   age   sex localization
0  HAM_0000118  ISIC_0027419  bkl   histo  80.0  male        scalp
1  HAM_0000118  ISIC_0025030  bkl   histo  80.0  male        scalp
2  HAM_0002730  ISIC_0026769  bkl   histo  80.0  male        scalp
3  HAM_0002730  ISIC_0025661  bkl   histo  80.0  male        scalp
4  HAM_0001466  ISIC_0031633  bkl   histo  75.0  male          ear
7010 3005
1502 1503


Image Processing + Input to Model

In [15]:
ham10000_pt1 = glob('HAM10000/HAM10000_part1/*.jpg')
ham1000_pt2 = glob('HAM10000/HAM10000_part2/*.jpg')

pixel_rep = []

#do it in batches of 1000

for path in ham10000_pt1:
    pixel_rep.append(plt.imread(path))

for path in ham1000_pt2:
    pixel_rep.append(plt.imread(path))


print("-----Results of reading in the image with imread-----")
print("First item in the array:")
print(pixel_rep[0])

# print(img_test)
# print(ham1000_pt2[0])
# print(ham10000_pt1[0])

-----Results of reading in the image with imread-----
First item in the array:
[[[221 131 131]
  [222 128 126]
  [223 129 127]
  ...
  [229 134 138]
  [226 136 138]
  [230 135 139]]

 [[223 129 129]
  [225 130 126]
  [228 132 133]
  ...
  [225 135 137]
  [226 136 136]
  [226 134 135]]

 [[222 128 126]
  [229 129 131]
  [231 130 134]
  ...
  [224 136 132]
  [227 136 133]
  [228 137 134]]

 ...

 [[209 118 125]
  [210 119 126]
  [212 121 126]
  ...
  [210 140 128]
  [209 141 130]
  [207 139 128]]

 [[205 116 118]
  [208 119 121]
  [209 120 124]
  ...
  [210 142 129]
  [212 144 131]
  [211 143 130]]

 [[207 116 115]
  [209 120 116]
  [208 120 118]
  ...
  [208 143 125]
  [211 146 126]
  [209 144 124]]]


In [ ]:
print(len(pixel_rep))

#pd.DataFrame(pixel_rep, columns=['image'])

10015


ValueError: Must pass 2-d input. shape=(10015, 450, 600, 3)

In [ ]:
#diide each RGB by 255 to get numbers in range 0 - 1